In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
mem_data = pd.read_csv('kaggle_data/mem_data.csv')
mem_tr = pd.read_csv('kaggle_data/mem_transaction.csv')
s_info = pd.read_csv('kaggle_data/store_info.csv')

In [3]:
mem_data.head()

,MEM_ID,M_STORE_ID,GENDER,BIRTH_DT,BIRTH_SL,ZIP_CD,RGST_DT,VISIT_CNT,SALES_AMT,LAST_VST_DT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS
0,1134945,1084,M,NaN,S,-,2006-02-04 11:51:08.233000000,8.0,337000.0,2007-01-19 21:46:12.827000000,6740.0,0.0,6740.0,3202.0,Y
1,38458,539,F,1973-07-11,S,420-721,2006-02-04 11:51:08.233000000,22.0,1438500.0,2007-10-18 19:57:45.503000000,1732.0,12000.0,13732.0,17762.0,Y
2,7009,1113,F,1973-04-06,S,-,2006-02-04 11:51:08.233000000,8.0,309300.0,2007-05-30 15:21:10.863000000,6372.0,0.0,6372.0,1856.0,Y
3,91791,2273,UNKNOWN,1968-09-05,S,487-820,2006-02-04 11:51:08.233000000,11.0,170400.0,2007-10-16 15:55:52.483000000,3408.0,0.0,3408.0,236.0,Y
4,1374842,1300,M,NaN,S,-,2006-02-04 11:51:08.233000000,3.0,110300.0,2007-03-25 12:49:30.170000000,2206.0,0.0,2206.0,1522.0,Y


In [4]:
mem_data.GENDER.value_counts()

M          3767
F          3767
UNKNOWN    3230
Name: GENDER, dtype: int64

In [5]:
mem_data['BIRTH_DT'].isnull().sum()

4767

### SMS 수신동의 정수 처리하기

In [6]:
mem_data.SMS = (mem_data.SMS=='Y').astype(int)
mem_data.SMS.value_counts()

1    10575
0      189
Name: SMS, dtype: int64

### 양/음력(BIRTH_SL) 정수 처리하기

In [7]:
mem_data.BIRTH_SL = (mem_data.BIRTH_SL=='L').astype(int)
mem_data.BIRTH_SL.value_counts()

0    8973
1    1791
Name: BIRTH_SL, dtype: int64

### 구매 합계(SALES_AMT) 로그 처리 하기

In [8]:
import math
from sklearn import preprocessing

In [9]:
# mem_data['SALES_AMT'] = mem_data['SALES_AMT'].abs()
# mem_data['SALES_AMT_log'] = preprocessing.scale(np.log(mem_data['SALES_AMT']+1))

In [10]:
f = mem_data.SALES_AMT.where(mem_data.SALES_AMT>=0, other=0) # 음수처리
f = np.log(f+1)
mem_data.SALES_AMT = f

In [11]:
mem_data['SALES_AMT'].describe()

count    10764.000000
mean        11.345458
std          1.078276
min          0.000000
25%         10.696503
50%         11.378262
75%         12.048401
max         18.420681
Name: SALES_AMT, dtype: float64

In [12]:
mem_data['ACC_PNT'].describe()

count    1.076400e+04
mean     3.229333e+03
std      2.287569e+04
min      0.000000e+00
25%      9.460000e+02
50%      1.844000e+03
75%      3.504000e+03
max      2.000000e+06
Name: ACC_PNT, dtype: float64

### 최근 방문 일자(LAST_VST_DT)로부터 경과일 구하기

In [13]:
f = pd.to_datetime(mem_data.LAST_VST_DT)
mem_data['L_DAY'] = (pd.to_datetime('2007-12-13') - f).dt.days
mem_data['L_DAY'].head()

0    327
1     55
2    196
3     57
4    262
Name: L_DAY, dtype: int64

### 등록일(RGST_DT)로부터 경과일 구하기

In [14]:
f = pd.to_datetime(mem_data.RGST_DT)
mem_data['R_DAY'] = (pd.to_datetime('2007-12-13') - f).dt.days
mem_data['R_DAY'].head()

0    676
1    676
2    676
3    676
4    676
Name: R_DAY, dtype: int64

In [15]:
f = mem_tr.groupby('MEM_ID')['SELL_AMT'].agg({'mean'}).reset_index()
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

### 주중 / 주말 구매패턴 구하기

In [16]:
mem_tr['SELL_DT'] = mem_tr['SELL_DT'].astype(str)
mem_tr['SELL_DT'] = pd.to_datetime(mem_tr['SELL_DT'])
mem_tr['SELL_DT_weekday'] = mem_tr['SELL_DT'].dt.weekday

In [17]:
# 0 = 주중, 1 = 주말
mem_tr['SELL_DT_weekday'] = mem_tr['SELL_DT_weekday'].apply(lambda x: 0 if x < 5
                                                                    else 1)

In [18]:
mem_tr['SELL_DT_weekday'].value_counts()

0    40191
1    19858
Name: SELL_DT_weekday, dtype: int64

In [19]:
f = mem_tr.groupby('MEM_ID')['SELL_DT_weekday'].agg({'sum'}).reset_index()
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

In [20]:
f = mem_tr.groupby('MEM_ID')['SELL_DT_weekday'].agg({'count'}).reset_index()
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

In [21]:
mem_data.head()

,MEM_ID,M_STORE_ID,GENDER,BIRTH_DT,BIRTH_SL,ZIP_CD,RGST_DT,VISIT_CNT,SALES_AMT,LAST_VST_DT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,L_DAY,R_DAY,mean,sum,count
0,1134945,1084,M,NaN,0,-,2006-02-04 11:51:08.233000000,8.0,12.727841,2007-01-19 21:46:12.827000000,6740.0,0.0,6740.0,3202.0,1,327,676,16262.500000,0,8
1,38458,539,F,1973-07-11,0,420-721,2006-02-04 11:51:08.233000000,22.0,14.179112,2007-10-18 19:57:45.503000000,1732.0,12000.0,13732.0,17762.0,1,55,676,25018.181818,3,22
2,7009,1113,F,1973-04-06,0,-,2006-02-04 11:51:08.233000000,8.0,12.642070,2007-05-30 15:21:10.863000000,6372.0,0.0,6372.0,1856.0,1,196,676,22025.000000,2,8
3,91791,2273,UNKNOWN,1968-09-05,0,487-820,2006-02-04 11:51:08.233000000,11.0,12.045910,2007-10-16 15:55:52.483000000,3408.0,0.0,3408.0,236.0,1,57,676,14418.181818,2,11
4,1374842,1300,M,NaN,0,-,2006-02-04 11:51:08.233000000,3.0,11.610968,2007-03-25 12:49:30.170000000,2206.0,0.0,2206.0,1522.0,1,262,676,11400.000000,2,3


In [22]:
mem_data['Week_Visit_Rate'] = mem_data['sum'] / mem_data['count']
mem_data

,MEM_ID,M_STORE_ID,GENDER,BIRTH_DT,BIRTH_SL,ZIP_CD,RGST_DT,VISIT_CNT,SALES_AMT,LAST_VST_DT,...,USED_PNT,ACC_PNT,USABLE_INIT,SMS,L_DAY,R_DAY,mean,sum,count,Week_Visit_Rate
0,1134945,1084,M,NaN,0,-,2006-02-04 11:51:08.233000000,8.0,12.727841,2007-01-19 21:46:12.827000000,...,0.0,6740.0,3202.0,1,327,676,16262.500000,0,8,0.000000
1,38458,539,F,1973-07-11,0,420-721,2006-02-04 11:51:08.233000000,22.0,14.179112,2007-10-18 19:57:45.503000000,...,12000.0,13732.0,17762.0,1,55,676,25018.181818,3,22,0.136364
2,7009,1113,F,1973-04-06,0,-,2006-02-04 11:51:08.233000000,8.0,12.642070,2007-05-30 15:21:10.863000000,...,0.0,6372.0,1856.0,1,196,676,22025.000000,2,8,0.250000
3,91791,2273,UNKNOWN,1968-09-05,0,487-820,2006-02-04 11:51:08.233000000,11.0,12.045910,2007-10-16 15:55:52.483000000,...,0.0,3408.0,236.0,1,57,676,14418.181818,2,11,0.181818
4,1374842,1300,M,NaN,0,-,2006-02-04 11:51:08.233000000,3.0,11.610968,2007-03-25 12:49:30.170000000,...,0.0,2206.0,1522.0,1,262,676,11400.000000,2,3,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10759,1363530,2117,UNKNOWN,NaN,0,-,2006-02-04 11:51:08.233000000,2.0,12.523802,2006-12-12 21:33:19.477000000,...,0.0,5496.0,440.0,1,365,676,126400.000000,1,2,0.500000
10760,684874,997,UNKNOWN,1989-02-02,0,-,2006-02-04 11:51:08.233000000,2.0,10.581546,2007-06-20 15:19:00.280000000,...,0.0,964.0,314.0,1,175,676,11850.000000,0,2,0.000000
10761,454881,1181,M,NaN,0,-,2006-02-04 11:51:08.233000000,3.0,11.359786,2007-04-14 21:13:18.977000000,...,0.0,1716.0,0.0,1,242,676,28600.000000,1,3,0.333333
10762,973807,812,M,NaN,0,-,2006-02-04 11:51:08.233000000,4.0,12.246259,2007-09-12 12:58:43.683000000,...,0.0,4164.0,1110.0,1,91,676,38175.000000,1,4,0.250000


In [23]:
del_col = ['BIRTH_DT', 'ZIP_CD', 'RGST_DT', 'LAST_VST_DT', 'sum', 'count']
mem_data = mem_data.drop(del_col, axis = 1)
mem_data.head()

,MEM_ID,M_STORE_ID,GENDER,BIRTH_SL,VISIT_CNT,SALES_AMT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,L_DAY,R_DAY,mean,Week_Visit_Rate
0,1134945,1084,M,0,8.0,12.727841,6740.0,0.0,6740.0,3202.0,1,327,676,16262.500000,0.000000
1,38458,539,F,0,22.0,14.179112,1732.0,12000.0,13732.0,17762.0,1,55,676,25018.181818,0.136364
2,7009,1113,F,0,8.0,12.642070,6372.0,0.0,6372.0,1856.0,1,196,676,22025.000000,0.250000
3,91791,2273,UNKNOWN,0,11.0,12.045910,3408.0,0.0,3408.0,236.0,1,57,676,14418.181818,0.181818
4,1374842,1300,M,0,3.0,11.610968,2206.0,0.0,2206.0,1522.0,1,262,676,11400.000000,0.666667


In [24]:
tr = mem_data.GENDER!='UNKNOWN'
train = mem_data[tr]
train.GENDER = (train.GENDER=='M').astype(int)
train

C:\Users\TJ\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,MEM_ID,M_STORE_ID,GENDER,BIRTH_SL,VISIT_CNT,SALES_AMT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,L_DAY,R_DAY,mean,Week_Visit_Rate
0,1134945,1084,1,0,8.0,12.727841,6740.0,0.0,6740.0,3202.0,1,327,676,16262.500000,0.000000
1,38458,539,0,0,22.0,14.179112,1732.0,12000.0,13732.0,17762.0,1,55,676,25018.181818,0.136364
2,7009,1113,0,0,8.0,12.642070,6372.0,0.0,6372.0,1856.0,1,196,676,22025.000000,0.250000
4,1374842,1300,1,0,3.0,11.610968,2206.0,0.0,2206.0,1522.0,1,262,676,11400.000000,0.666667
6,1383660,1024,1,0,2.0,10.419331,670.0,0.0,670.0,110.0,1,459,675,14000.000000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10756,343892,130,0,0,22.0,12.856576,7866.0,0.0,7866.0,2884.0,1,45,676,10718.181818,0.500000
10757,312177,130,0,0,18.0,12.863077,870.0,7000.0,7870.0,4292.0,1,35,676,9511.111111,0.166667
10761,454881,1181,1,0,3.0,11.359786,1716.0,0.0,1716.0,0.0,1,242,676,28600.000000,0.333333
10762,973807,812,1,0,4.0,12.246259,4164.0,0.0,4164.0,1110.0,1,91,676,38175.000000,0.250000


In [25]:
ne = mem_data.GENDER=='UNKNOWN'
new = mem_data[ne].sort_values('MEM_ID')
new.head()

,MEM_ID,M_STORE_ID,GENDER,BIRTH_SL,VISIT_CNT,SALES_AMT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,L_DAY,R_DAY,mean,Week_Visit_Rate
9082,1054,550,UNKNOWN,1,5.0,12.358369,29158.0,0.0,29158.0,846.0,0,57,676,38120.0,0.0
7336,1231,550,UNKNOWN,0,1.0,11.229249,1506.0,0.0,1506.0,1418.0,1,627,676,4400.0,1.0
6678,1400,550,UNKNOWN,0,1.0,12.038843,3384.0,0.0,3384.0,3088.0,0,477,676,3000.0,0.0
5774,1483,457,UNKNOWN,0,1.0,11.591746,2214.0,0.0,2214.0,936.0,1,339,676,61400.0,1.0
1756,1581,550,UNKNOWN,0,2.0,11.226589,1502.0,0.0,1502.0,1174.0,1,146,676,8200.0,0.0


In [26]:
train.corr()

,MEM_ID,M_STORE_ID,GENDER,BIRTH_SL,VISIT_CNT,SALES_AMT,USABLE_PNT,USED_PNT,ACC_PNT,USABLE_INIT,SMS,L_DAY,R_DAY,mean,Week_Visit_Rate
MEM_ID,1.000000,-0.008683,-0.012590,0.003406,0.057345,0.014700,0.044993,0.027781,0.045750,-0.033789,-0.244340,-0.019564,-0.856224,0.026485,-0.007705
M_STORE_ID,-0.008683,1.000000,-0.042707,0.071338,0.011109,0.025411,0.011690,0.001015,0.007016,0.016987,0.003860,-0.023035,0.011581,-0.011863,0.026051
GENDER,-0.012590,-0.042707,1.000000,-0.357058,-0.061362,-0.171529,-0.073629,-0.026084,-0.059766,-0.160997,-0.028540,0.133481,0.003956,0.019697,0.024099
BIRTH_SL,0.003406,0.071338,-0.357058,1.000000,0.033658,0.065047,0.031521,0.023625,0.035849,0.031861,0.020901,-0.052539,-0.006120,0.015101,-0.005072
VISIT_CNT,0.057345,0.011109,-0.061362,0.033658,1.000000,0.466714,0.668223,0.444101,0.704506,0.394452,-0.059416,-0.346891,-0.021111,0.030509,-0.022230
SALES_AMT,0.014700,0.025411,-0.171529,0.065047,0.466714,1.000000,0.522858,0.320215,0.528478,0.557621,-0.001767,-0.431757,0.021520,0.283016,-0.008596
USABLE_PNT,0.044993,0.011690,-0.073629,0.031521,0.668223,0.522858,1.000000,0.170058,0.692579,0.508045,-0.061900,-0.235860,-0.028741,0.218867,-0.004927
USED_PNT,0.027781,0.001015,-0.026084,0.023625,0.444101,0.320215,0.170058,1.000000,0.826988,0.278558,-0.004291,-0.119028,-0.008885,0.297869,-0.013608
ACC_PNT,0.045750,0.007016,-0.059766,0.035849,0.704506,0.528478,0.692579,0.826988,1.000000,0.490077,-0.037970,-0.220919,-0.022861,0.342327,-0.011967
USABLE_INIT,-0.033789,0.016987,-0.160997,0.031861,0.394452,0.557621,0.508045,0.278558,0.490077,1.000000,-0.014035,-0.125579,0.044167,0.152785,-0.007577


In [30]:
from sklearn.model_selection import train_test_split

In [27]:
dfX = train.drop(['MEM_ID','GENDER'], axis=1)
dfy = train['GENDER']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(dfX, dfy, test_size=0.25, random_state=0)

In [32]:
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier(n_estimators=200, random_state=0)
gbm.fit(X_train, y_train).score(X_test, y_test)

0.8646496815286624

In [33]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(n_estimators=200, random_state=0)
ada.fit(X_train, y_train).score(X_test, y_test)

0.8434182590233545

In [36]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=4, random_state=0)
tree.fit(X_train, y_train).score(X_test, y_test)

0.778131634819533

In [37]:
!pip install lightgbm

In [38]:
import lightgbm as lgb
train_ds = lgb.Dataset(X_train, label = y_train)
test_ds = lgb.Dataset(X_val. label = y_val)
params = {'learning_rate' : 0.01,
         'max_depth':16,
         'boosting':'gbdt',
         'objective':'regression',
         'metric':'mse',
          'is_training_metric':True
          'num_leaves':144,
          'feature_fraction':0.9,
          'bagging_fraction':0.7,
          'bagging_freq':5,
          'seed':2020}

model = lgb.train(params, train_ds, 1000, test_ds, verbose_eval=100, early_stopping_rounds=100)
y_pred=model.predict(X_val)

SyntaxError: invalid syntax (<ipython-input-38-c1e1cf1080a8>, line 10)